Código

In [98]:
import json
from tabulate import tabulate
from datetime import datetime
from collections import deque
import uuid

exames = {}


def gerar_id_exame():
    return str(uuid.uuid4())[:4]  

def inserir_exame(id_paciente, nome, data, tipo_peca, ia_altura, ia_largura, ia_peso, real_altura, real_largura, real_peso):
    if id_paciente not in exames:
        exames[id_paciente] = {
            "nome": nome,
            "exames": []
        }

    novo_exame = {
        "id_exame": gerar_id_exame(),
        "data": data,
        "tipo_peca": tipo_peca,
        "IA_altura": ia_altura,
        "IA_largura": ia_largura,
        "IA_peso": ia_peso,
        "Real_altura": real_altura,
        "Real_largura": real_largura,
        "Real_peso": real_peso
    }

    exames[id_paciente]["exames"].append(novo_exame)

def exibir_tabela(exames_dict):
    linhas = []
    for id_paciente, dados in exames_dict.items():
        for exame in dados["exames"]:
            linha = [
                id_paciente,
                dados["nome"],
                exame["id_exame"],
                exame["data"],
                exame["tipo_peca"],
                exame["IA_altura"],
                exame["IA_largura"],
                exame["IA_peso"],
                exame["Real_altura"],
                exame["Real_largura"],
                exame["Real_peso"]
            ]
            linhas.append(linha)
    headers = ["ID Paciente", "Nome", "ID Exame", "Data", "Tipo Peça", "IA Altura", "IA Largura", "IA Peso",
               "Real Altura", "Real Largura", "Real Peso"]
    print(tabulate(linhas, headers=headers, tablefmt="grid"))

def bubble_sort(lista):
    n = len(lista)
    for i in range(n):
        for j in range(0, n - i - 1):
            nome_atual = lista[j][1]["nome"].lower()
            nome_proximo = lista[j + 1][1]["nome"].lower()
            if nome_atual > nome_proximo:
                lista[j], lista[j + 1] = lista[j + 1], lista[j]
    return lista

def bubble_sort_exame(lista):
    n = len(lista)
    for i in range(n):
        for j in range(0, n - i - 1):
            if lista[j]["id_exame"] > lista[j + 1]["id_exame"]:
                lista[j], lista[j + 1] = lista[j + 1], lista[j]
    return lista

def busca_binaria_exame(lista, id_alvo):
    inicio = 0
    fim = len(lista) - 1
    while inicio <= fim:
        meio = (inicio + fim) // 2
        if lista[meio]["id_exame"] == id_alvo:
            return meio
        elif lista[meio]["id_exame"] < id_alvo:
            inicio = meio + 1
        else:
            fim = meio - 1
    return -1

def excluir_exame_por_id(id_exame):
    for id_paciente in list(exames.keys()):
        exames_paciente = exames[id_paciente]["exames"]
        exames_ordenados = bubble_sort_exame(exames_paciente.copy())  
        indice = busca_binaria_exame(exames_ordenados, id_exame)

        if indice != -1:
            for i, exame in enumerate(exames_paciente):
                if exame["id_exame"] == id_exame:
                    exames_paciente.pop(i)
                    break

            if not exames_paciente:
                del exames[id_paciente]
            print(f"Exame com ID {id_exame} excluído com sucesso.")
            return
    print(f"Exame com ID {id_exame} não encontrado.")





def extrair_exames_paciente(id_paciente):
    pacientes_ordenados = bubble_sort(list(exames.items()))
    ids_ordenados = [item[0] for item in pacientes_ordenados]

    
    inicio, fim = 0, len(ids_ordenados) - 1
    while inicio <= fim:
        meio = (inicio + fim) // 2
        if ids_ordenados[meio] == id_paciente:
            return {id_paciente: dict(pacientes_ordenados[meio][1])}
        elif ids_ordenados[meio] < id_paciente:
            inicio = meio + 1
        else:
            fim = meio - 1

    return None 

def salvar_dados(dados_json, nome_arquivo):
    try:
        with open(nome_arquivo, 'w', encoding='utf-8') as f:
            json.dump(dados_json, f, ensure_ascii=False, indent=4)
        print(f"Arquivo '{nome_arquivo}' salvo com sucesso.")
    except Exception as e:
        print(f"Erro ao salvar o arquivo: {e}")


def historico_exames_recente(id_paciente, n=1):
    if id_paciente not in exames:
        print("Paciente não encontrado.")
        return {}

    exames_paciente = exames[id_paciente]["exames"]
    
    # Ordenar usando método clássico (exemplo: insertion sort)
    for i in range(1, len(exames_paciente)):
        key = exames_paciente[i]
        j = i - 1
        while j >= 0 and datetime.strptime(exames_paciente[j]["data"], "%Y-%m-%d") < datetime.strptime(key["data"], "%Y-%m-%d"):
            exames_paciente[j + 1] = exames_paciente[j]
            j -= 1
        exames_paciente[j + 1] = key

    return {
        id_paciente: {
            "nome": exames[id_paciente]["nome"],
            "exames": exames_paciente[:n]
        }
    }


fila_analise = {
    "Alta": deque(),
    "Media": deque(),
    "Baixa": deque()
}

def adicionar_na_fila_analise(id_paciente, nome, tipo_peca, prioridade):
    prioridade = prioridade.capitalize()
    if prioridade not in fila_analise:
        print("Prioridade inválida. Use: Alta, Media ou Baixa.")
        return
    
    fila_analise[prioridade].append({
        "id": id_paciente,
        "nome": nome,
        "tipo_peca": tipo_peca,
        "prioridade": prioridade
    })
    print(f"Adicionado à fila ({prioridade}): {nome} - {tipo_peca}")

def fila_espera_analise():
    print("Fila de Espera para Análise (por prioridade):")
    
    for prioridade in ["Alta", "Media", "Baixa"]:
        print(f"\nPrioridade: {prioridade}")
        for item in fila_analise[prioridade]:
            print(f"  • ID: {item['id']}, Nome: {item['nome']}, Tipo: {item['tipo_peca']}")

IMPLEMENTAÇÃO

In [99]:
# Inserindo dados para teste do algoritimo

inserir_exame("001", "Alice", "2025-05-01", "Pele", 10.2, 5.1, 3.3, 9.8, 4.9, 3.1)
inserir_exame("001", "Alice", "2025-05-15", "Rim", 120.0, 65.0, 150.5, 118.5, 63.7, 148.9) 
inserir_exame("002", "Bruno", "2025-04-20", "Pele", 9.5, 4.8, 3.0, 9.0, 4.5, 2.9)
inserir_exame("003", "Carla", "2025-03-10", "Pinta", 8.7, 4.4, 0.5, 8.3, 4.0, 0.4) 
inserir_exame("004", "Daniel", "2025-05-10", "Biópsia de Próstata", 15.5, 3.2, 1.2, 15.0, 3.0, 1.1)
inserir_exame("001", "Alice", "2025-06-01", "Pinta", 6.3, 3.1, 0.3, 6.0, 2.9, 0.25)
inserir_exame("002", "Bruno", "2025-05-25", "Nódulo Tireoide", 20.1, 15.8, 5.5, 19.5, 15.2, 5.3)
inserir_exame("005", "Eduarda", "2025-04-28", "Pele", 12.8, 6.5, 4.0, 12.2, 6.1, 3.8)
inserir_exame("003", "Carla", "2025-05-18", "Fragmento Ósseo", 25.0, 10.0, 8.5, 24.3, 9.5, 8.1)
inserir_exame("006", "Fernando", "2025-03-01", "Pólipo Intestinal", 11.5, 7.0, 1.8, 11.0, 6.8, 1.7)
inserir_exame("004", "Daniel", "2025-06-10", "Pele", 7.9, 3.5, 1.0, 7.5, 3.3, 0.9)
inserir_exame("001", "Alice", "2025-06-20", "Biópsia Endometrial", 18.0, 4.0, 2.1, 17.5, 3.8, 2.0)
inserir_exame("005", "Eduarda", "2025-06-05", "Pinta", 5.5, 2.8, 0.2, 5.1, 2.5, 0.18)
inserir_exame("007", "Gabriela", "2025-02-15", "Rim", 115.0, 60.0, 140.0, 113.2, 58.5, 138.5)
inserir_exame("002", "Bruno", "2025-06-30", "Linfonodo", 13.5, 9.2, 3.5, 13.0, 8.9, 3.3)
inserir_exame("008", "Heitor", "2025-01-20", "Amostra de Fígado", 30.5, 20.0, 25.6, 29.8, 19.5, 25.0)
inserir_exame("006", "Fernando", "2025-04-15", "Pele", 14.2, 7.1, 4.5, 13.8, 6.8, 4.3)
inserir_exame("003", "Carla", "2025-06-25", "Cisto Sebáceo", 19.0, 18.5, 6.2, 18.5, 18.0, 6.0)
inserir_exame("009", "Isabela", "2025-05-05", "Vesícula Biliar", 70.0, 30.0, 45.0, 68.5, 28.0, 43.5)
inserir_exame("001", "Alice", "2025-07-01", "Fragmento Muscular", 22.5, 8.0, 5.8, 22.0, 7.7, 5.5)
inserir_exame("010", "João", "2025-02-10", "Pele", 11.0, 5.5, 3.5, 10.5, 5.2, 3.3)

# Exibindo Todos os registros 

In [100]:

print("Todos os exames:")
exibir_tabela(exames)

Todos os exames:
+---------------+----------+------------+------------+---------------------+-------------+--------------+-----------+---------------+----------------+-------------+
|   ID Paciente | Nome     | ID Exame   | Data       | Tipo Peça           |   IA Altura |   IA Largura |   IA Peso |   Real Altura |   Real Largura |   Real Peso |
+===============+==========+============+============+=====================+=============+==============+===========+===============+================+=============+
|           001 | Alice    | ac6f       | 2025-05-01 | Pele                |        10.2 |          5.1 |       3.3 |           9.8 |            4.9 |        3.1  |
+---------------+----------+------------+------------+---------------------+-------------+--------------+-----------+---------------+----------------+-------------+
|           001 | Alice    | d330       | 2025-05-15 | Rim                 |       120   |         65   |     150.5 |         118.5 |           63.7 |      14

# Buscando\Extraindo e Exibindo os 1 Paciente

In [101]:
paciente = '001'
print(f"Exames do paciente {paciente}:")
exames_paciente= extrair_exames_paciente(paciente)
if exames_paciente:
    exibir_tabela(exames_paciente)
else:
    print("Paciente não encontrado.")


Exames do paciente 001:
+---------------+--------+------------+------------+---------------------+-------------+--------------+-----------+---------------+----------------+-------------+
|   ID Paciente | Nome   | ID Exame   | Data       | Tipo Peça           |   IA Altura |   IA Largura |   IA Peso |   Real Altura |   Real Largura |   Real Peso |
+===============+========+============+============+=====================+=============+==============+===========+===============+================+=============+
|           001 | Alice  | ac6f       | 2025-05-01 | Pele                |        10.2 |          5.1 |       3.3 |           9.8 |            4.9 |        3.1  |
+---------------+--------+------------+------------+---------------------+-------------+--------------+-----------+---------------+----------------+-------------+
|           001 | Alice  | d330       | 2025-05-15 | Rim                 |       120   |         65   |     150.5 |         118.5 |           63.7 |      148.9  

# Extraindo/Exibindo o Historico do N Exames Mais Recentes de um Paciente

In [102]:
n_exames = 3
paciente = '001'
print(f"Os {n_exames} exames mais recentes da paciente {paciente}:")
recentes = historico_exames_recente(paciente, n_exames)
exibir_tabela(recentes)

Os 3 exames mais recentes da paciente 001:
+---------------+--------+------------+------------+---------------------+-------------+--------------+-----------+---------------+----------------+-------------+
|   ID Paciente | Nome   |   ID Exame | Data       | Tipo Peça           |   IA Altura |   IA Largura |   IA Peso |   Real Altura |   Real Largura |   Real Peso |
+===============+========+============+============+=====================+=============+==============+===========+===============+================+=============+
|           001 | Alice  |       7392 | 2025-07-01 | Fragmento Muscular  |        22.5 |          8   |       5.8 |          22   |            7.7 |        5.5  |
+---------------+--------+------------+------------+---------------------+-------------+--------------+-----------+---------------+----------------+-------------+
|           001 | Alice  |       2592 | 2025-06-20 | Biópsia Endometrial |        18   |          4   |       2.1 |          17.5 |           

# Exibindo (Fila) de espera para fazer um exame

In [103]:
adicionar_na_fila_analise("001", "Alice", "Tumor", "Alta")
adicionar_na_fila_analise("002", "Bruno", "Biópsia", "Media")
adicionar_na_fila_analise("003", "Carlos", "Lesão", "Baixa")
adicionar_na_fila_analise("004", "Diana", "Nódulo", "Alta")



Adicionado à fila (Alta): Alice - Tumor
Adicionado à fila (Media): Bruno - Biópsia
Adicionado à fila (Baixa): Carlos - Lesão
Adicionado à fila (Alta): Diana - Nódulo


In [104]:
fila_espera_analise()

Fila de Espera para Análise (por prioridade):

Prioridade: Alta
  • ID: 001, Nome: Alice, Tipo: Tumor
  • ID: 004, Nome: Diana, Tipo: Nódulo

Prioridade: Media
  • ID: 002, Nome: Bruno, Tipo: Biópsia

Prioridade: Baixa
  • ID: 003, Nome: Carlos, Tipo: Lesão


# Excluindo exames do histórico

In [105]:
id_exame = 'dca5'
excluir_exame_por_id(id_exame)

Exame com ID dca5 não encontrado.


# Salvando Dados Json

In [106]:
salvar_dados(exames, "exames_pacientes.json")


Arquivo 'exames_pacientes.json' salvo com sucesso.
